In [1]:
import torch
from torch import nn
from mixture_of_experts import MoE
import numpy as np
import random
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

In [19]:
class MoE_Projection(nn.Module):
    def __init__(self, dim, hidden_dim, num_experts, qkv_dim):
        super().__init__()
        self.moe = MoE(
            dim = dim,
            num_experts = num_experts,               # increase the experts (# parameters) of your model without increasing computation
            hidden_dim = hidden_dim,           # size of hidden dimension in each expert, defaults to 4 * dimension
            activation = nn.LeakyReLU,      # use your preferred activation, will default to GELU
            second_policy_train = 'random', # in top_2 gating, policy for whether to use a second-place expert
            second_policy_eval = 'random',  # all (always) | none (never) | threshold (if gate value > the given threshold) | random (if gate value > threshold * random_uniform(0, 1))
            second_threshold_train = 0.2,
            second_threshold_eval = 0.2,
            capacity_factor_train = 1.25,   # experts have fixed capacity per batch. we need some extra capacity in case gating is not perfectly balanced.
            capacity_factor_eval = 2.,      # capacity_factor_* should be set to a value >=1
            loss_coef = 1e-2                # multiplier on the auxiliary expert balancing auxiliary loss
        )
        self.norm = nn.LayerNorm(dim)
        self.to_qkv = nn.Linear(dim, qkv_dim * 3, bias = False)

    def forward(self, x):
        x, aux_loss = self.moe(x)
        x = self.norm(x)
        qkv = list(self.to_qkv(x).chunk(3, dim = -1))
        
        return x, qkv, aux_loss


In [26]:
class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.norm = nn.LayerNorm(dim)

        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, qkv):
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

In [27]:
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [32]:
def qurey_fusion(qkv_list, idx):
    
    # qkv_list: [(q_1, k_1, v_1), (q_2, k_2, v_2), ...]
    for i in range(len(qkv_list)):
        # print(qkv_list[i][0].shape)
        qkv_list[i][0] = qkv_list[idx][0]
    
    return qkv_list

In [42]:
def fuision_idx(num_modalities, model_depth):
    
    " generate qurey_fusion_idx "
    qurey_fusion_idx = []
    repeat_list = list(range(num_modalities))
    for _ in range(model_depth):
        idx = random.randint(0,num_modalities-1)
        while idx not in repeat_list:
            idx = random.randint(0,num_modalities-1)
        repeat_list.remove(idx)
        qurey_fusion_idx.append(idx)
        if len(repeat_list) == 0:
            repeat_list = list(range(num_modalities))
    
    return qurey_fusion_idx

In [34]:
class fusion_transformer(nn.Module):
    def __init__(self, dim, hidden_dim, num_experts, qkv_dim, num_heads, dim_heads):
        super().__init__()
        self.layers = nn.ModuleList([])
        self.moe = MoE_Projection(dim, hidden_dim, num_experts, qkv_dim)
        # self.norm = nn.LayerNorm(dim)
        # self.dropout = nn.Dropout(dropout)
        self.mutihead_attention = Attention(dim, num_heads, dim_heads)
        self.feed_forward = FeedForward(dim, hidden_dim)
    def forward(self, x, modality_list, qurey_fusion_idx):
        num_modalities = sum(modality_list)
        
        features = list(x.chunk(num_modalities, dim = -2))
        qkv_list = []
        aux_loss_list = []
        for i in range(num_modalities):
            features[i], qkv, aux_loss = self.moe(features[i])
            qkv_list.append(qkv)
            aux_loss_list.append(aux_loss)
        fused_qkv_list = qurey_fusion(qkv_list, qurey_fusion_idx)
        for j in range(num_modalities):
            features[j] = self.mutihead_attention(fused_qkv_list[j]) + features[j]
            features[j] = self.feed_forward(features[j]) + features[j]
        x = torch.cat(features, dim = -2)
        return x

In [55]:
class regression_Head(nn.Sequential):
    def __init__(self, emb_size=512, num_classes=17*3):
        super().__init__()
        self.norm = nn.LayerNorm(emb_size)
        self.fc = nn.Linear(emb_size, num_classes)
    
    def forward(self, x):
        # print(x.shape)
        x = torch.mean(x, dim=1)
        # print(x.shape)
        x = self.norm(x)
        x = self.fc(x)
        x = x.view(x.size(0), 17, 3)
        return x

In [56]:
class MI_Transformer(nn.Module):
    def __init__(self, dim, hidden_dim, num_experts, qkv_dim, num_heads, dim_heads, depth):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(fusion_transformer(dim, hidden_dim, num_experts, qkv_dim, num_heads, dim_heads))
        self.regression_head = regression_Head(dim, 17*3)
        self.depth = depth

    def forward(self, x, modality_list):
        qurey_fusion_idx_ls = fuision_idx(sum(modality_list), self.depth)
        i = 0
        for layer in self.layers:
            qurey_fusion_idx = qurey_fusion_idx_ls[i]
            print(qurey_fusion_idx)
            x = layer(x, modality_list, qurey_fusion_idx)
            i += 1
        x = self.regression_head(x)
        return x
        

In [59]:
x = torch.randn(16, 160, 512)
modality_list = [True, True, True, True, True]
# qurey_fusion_idx = 1
# layer = fusion_transformer(
#     dim = 512, 
#     hidden_dim = 1024, 
#     num_experts = 8, 
#     qkv_dim = 512,
#     num_heads = 8,
#     dim_heads = 64
#     )
model = MI_Transformer(
    dim = 512, 
    hidden_dim = 1024, 
    num_experts = 8, 
    qkv_dim = 512,
    num_heads = 8,
    dim_heads = 64,
    depth = 7
    )
# out = layer(x, modality_list, qurey_fusion_idx)
out = model(x, modality_list)
print(out.shape)

4
0
3
1
2
2
0
torch.Size([16, 17, 3])


In [18]:
moe = MoE(
            dim = 512,
            num_experts = 16,               # increase the experts (# parameters) of your model without increasing computation
            hidden_dim = 512 * 4,           # size of hidden dimension in each expert, defaults to 4 * dimension
            activation = nn.LeakyReLU,      # use your preferred activation, will default to GELU
            second_policy_train = 'random', # in top_2 gating, policy for whether to use a second-place expert
            second_policy_eval = 'random',  # all (always) | none (never) | threshold (if gate value > the given threshold) | random (if gate value > threshold * random_uniform(0, 1))
            second_threshold_train = 0.2,
            second_threshold_eval = 0.2,
            capacity_factor_train = 1.25,   # experts have fixed capacity per batch. we need some extra capacity in case gating is not perfectly balanced.
            capacity_factor_eval = 2.,      # capacity_factor_* should be set to a value >=1
            loss_coef = 1e-2                # multiplier on the auxiliary expert balancing auxiliary loss
        )

In [8]:
inputs = torch.randn(4, 32, 512)
# out, aux_loss = moe(inputs) # (4, 1024, 512), (1,)
MoE_layer = MoE_Projection(512, 512, 16, 512)
out, qkv, aux_loss = MoE_layer(inputs) # (4, 1024, 512), (1,)
print(out.shape, qkv[0].shape, aux_loss)

torch.Size([4, 32, 512]) torch.Size([4, 32, 512]) tensor(0.0166, grad_fn=<MulBackward0>)


In [ ]:
def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.norm = nn.LayerNorm(dim)

        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        x = self.norm(x)

        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout),
                FeedForward(dim, mlp_dim, dropout = dropout)
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x

        return self.norm(x)

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Linear(dim, num_classes)

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)